In [ ]:
# | default_exp layers/latent_space

# Imports

In [ ]:
# | export


from typing import Literal

import torch
from torch import nn
from torch.distributions import Normal, kl_divergence

from vision_architectures.blocks.cnn import CNNBlock3D, CNNBlockConfig
from vision_architectures.docstrings import populate_docstring
from vision_architectures.utils.custom_base_model import CustomBaseModel, Field, model_validator
from vision_architectures.utils.rearrange import rearrange_channels

# Configs

In [ ]:
# | export


class LatentEncoderConfig(CNNBlockConfig):
    init_low_var: bool = Field(False, description="Whether to initialize weights such that output variance is low")

    @model_validator(mode="before")
    @classmethod
    def validate_before(cls, data: dict):
        data.setdefault("in_channels", data.get("dim"))
        data.setdefault("out_channels", data.get("latent_dim"))
        return data

    @property
    def dim(self):
        return self.in_channels

    @property
    def latent_dim(self):
        return self.out_channels


class LatentDecoderConfig(CNNBlockConfig):
    @model_validator(mode="before")
    @classmethod
    def validate_before(cls, data: dict):
        data.setdefault("in_channels", data.get("latent_dim"))
        data.setdefault("out_channels", data.get("dim"))
        return data

    @property
    def latent_dim(self):
        return self.in_channels

    @property
    def dim(self):
        return self.out_channels


class GaussianLatentSpaceConfig(CustomBaseModel):
    pass

# Architectures

In [ ]:
# | export


class LatentEncoder(nn.Module):
    def __init__(self, config: LatentEncoderConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = LatentEncoderConfig.model_validate(config | kwargs)

        latent_dim = self.config.latent_dim

        self.dim_mapper = CNNBlock3D(self.config, checkpointing_level)
        self.quant_conv_mu = nn.Conv3d(latent_dim, latent_dim, 1)
        self.quant_conv_log_var = nn.Conv3d(latent_dim, latent_dim, 1)

        if self.config.init_low_var:
            self.init_low_var()

    def init_low_var(self, bias_constant: float = -1.0):
        nn.init.normal_(self.quant_conv_log_var.weight, std=0.001)
        nn.init.constant_(self.quant_conv_log_var.bias, bias_constant)

    @populate_docstring
    def forward(
        self,
        x: torch.Tensor,
        prior_mu: torch.Tensor | None = None,
        prior_log_var: torch.Tensor | None = None,
        return_logvar: bool = False,
        channels_first: bool = True,
    ):
        """Get latent space representation of the input by mapping it to the latent dimension and then extracting the
        mean and standard deviation of the latent space. If a prior distribution is provided, the input is expected to
        predict the deviation from the prior. If it is not provided, one can think of it as the deviation from a
        standard normal distribution is being predicted. The output is the mean and standard deviation of the latent
        space.

        Args:
            x: The input feature tensor.
            prior_mu: The mean of the prior distribution. If None, it is assumed to be the mean of a standard normal
                distribution. Defaults to None.
            prior_log_var: The log-variance of the prior distribution. If None, it is assumed to be log-variance of a
                standard normal distribution. Defaults to None.
            return_logvar: Whether to return the log-variance too. Defaults to False.
            channels_first: {CHANNELS_FIRST_DOC}. Defaults to True.

        Returns:
            z_mu: The mean of the latent space.
            z_sigma: The standard deviation of the latent space.
        """
        # x: (b, [dim], z, y, x, [dim])
        # z_mu_prior: (b, [latent_dim], z, y, x, [latent_dim])
        # z_logvar_prior: (b, [latent_dim], z, y, x, [latent_dim])

        assert (
            prior_mu is None and prior_log_var is None or (prior_mu is not None and prior_log_var is not None)
        ), "If prior_mu or prior_log_var are provided, both must be provided."

        x = rearrange_channels(x, channels_first, True)
        # (b, dim, z, y, x)
        if prior_mu is not None:
            prior_mu = rearrange_channels(prior_mu, channels_first, True)
            prior_log_var = rearrange_channels(prior_log_var, channels_first, True)
            # (b, latent_dim, z, y, x)

        x = self.dim_mapper(x, channels_first=True)
        # (b, latent_dim, z, y, x)

        z_mu = self.quant_conv_mu(x)
        z_log_var = self.quant_conv_log_var(x)
        # (b, latent_dim, z, y, x)

        if prior_mu is not None:
            z_mu = prior_mu + z_mu
            z_log_var = prior_log_var + z_log_var
            # (b, latent_dim, z, y, x)

        z_log_var = torch.clamp(z_log_var, -10.0, 10.0)
        z_sigma = torch.exp(z_log_var / 2).clamp(min=1e-6)
        # (b, latent_dim, z, y, x)

        z_mu = rearrange_channels(z_mu, True, channels_first)
        z_sigma = rearrange_channels(z_sigma, True, channels_first)
        if return_logvar:
            z_log_var = rearrange_channels(z_log_var, True, channels_first)
        # (b, [latent_dim], z, y, x, [latent_dim])

        if return_logvar:
            return z_mu, z_sigma, z_log_var

        return z_mu, z_sigma

In [ ]:
test = LatentEncoder(dim=32, latent_dim=4, kernel_size=3)
display(test)

sample_input = torch.randn(2, 32, 16, 16, 16)
z_mu, z_sigma = test(sample_input)
z_mu.shape, z_sigma.shape


LatentEncoder(
  (dim_mapper): CNNBlock3D(
    (conv): Conv3d(32, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same, bias=False)
    (norm): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (checkpointing_level1): ActivationCheckpointing(enabled=False)
  )
  (quant_conv_mu): Conv3d(4, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (quant_conv_log_var): Conv3d(4, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1))
)

(torch.Size([2, 4, 16, 16, 16]), torch.Size([2, 4, 16, 16, 16]))

In [ ]:
test = LatentEncoder(dim=32, latent_dim=4, kernel_size=3)
display(test)

sample_input = torch.randn(2, 32, 16, 16, 16)
sample_prior_mu = torch.randn(2, 4, 16, 16, 16)
sample_prior_log_var = torch.randn(2, 4, 16, 16, 16)
z_mu, z_sigma = test(sample_input, sample_prior_mu, sample_prior_log_var)
z_mu.shape, z_sigma.shape


LatentEncoder(
  (dim_mapper): CNNBlock3D(
    (conv): Conv3d(32, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same, bias=False)
    (norm): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (checkpointing_level1): ActivationCheckpointing(enabled=False)
  )
  (quant_conv_mu): Conv3d(4, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (quant_conv_log_var): Conv3d(4, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1))
)

(torch.Size([2, 4, 16, 16, 16]), torch.Size([2, 4, 16, 16, 16]))

In [ ]:
# | export


class LatentDecoder(nn.Module):
    def __init__(self, config: LatentDecoderConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = LatentDecoderConfig.model_validate(config | kwargs)

        latent_dim = self.config.latent_dim

        self.post_quant_conv = nn.Conv3d(latent_dim, latent_dim, 1)
        self.dim_mapper = CNNBlock3D(self.config, checkpointing_level)

    def forward(self, z: torch.Tensor, channels_first: bool = True):
        # z: (b, [latent_dim], z, y, x, [latent_dim])

        z = rearrange_channels(z, channels_first, True)
        # (b, latent_dim, z, y, x)

        z = self.post_quant_conv(z)
        # (b, latent_dim, z, y, x)

        x = self.dim_mapper(z, channels_first=True)
        # (b, dim, z, y, x)

        x = rearrange_channels(x, True, channels_first)
        # (b, [dim], z, y, x, [dim])

        return x

In [ ]:
test = LatentDecoder(latent_dim=4, dim=32, kernel_size=3)
display(test)

sample_input = torch.randn(2, 4, 16, 16, 16)
test(sample_input).shape


LatentDecoder(
  (post_quant_conv): Conv3d(4, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (dim_mapper): CNNBlock3D(
    (conv): Conv3d(4, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same, bias=False)
    (norm): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (checkpointing_level1): ActivationCheckpointing(enabled=False)
  )
)

torch.Size([2, 32, 16, 16, 16])

In [ ]:
# | export


class GaussianLatentSpace(nn.Module):
    def __init__(self, config: GaussianLatentSpaceConfig = {}, **kwargs):
        super().__init__()

        self.config = GaussianLatentSpaceConfig.model_validate(config | kwargs)

    def sample(self, z_mu: torch.Tensor, z_sigma: torch.Tensor, force_sampling: bool = False):
        if not self.training and not force_sampling:
            return z_mu
        eps = torch.randn_like(z_sigma)
        z_vae = z_mu + eps * z_sigma
        return z_vae

    @staticmethod
    def kl_divergence(
        z_mu: torch.Tensor,
        z_sigma: torch.Tensor,
        prior_mu: torch.Tensor | None = None,
        prior_sigma: torch.Tensor | None = None,
        reduction: Literal["allsum", "channelssum"] | None = "allsum",
        channels_first: bool = True,
    ):
        if prior_mu is None:
            prior_mu = torch.zeros_like(z_mu)
        if prior_sigma is None:
            prior_sigma = torch.ones_like(z_sigma)

        prior = Normal(prior_mu, prior_sigma)
        posterior = Normal(z_mu, z_sigma)
        kl_div = kl_divergence(posterior, prior)

        if reduction is None:
            pass
        elif reduction == "allsum":
            kl_div = kl_div.sum(dim=list(range(1, kl_div.ndim))).mean()
        elif reduction == "channelssum":
            if channels_first:
                channels_dim = 1
            else:
                channels_dim = -1
            kl_div = kl_div.sum(dim=channels_dim).mean()
        else:
            raise NotImplementedError(f"Reduction {reduction} not implemented")

        return kl_div

    def forward(
        self,
        z_mu: torch.Tensor,
        z_sigma: torch.Tensor,
        prior_mu: torch.Tensor | None = None,
        prior_sigma: torch.Tensor | None = None,
        kl_divergence_reduction: Literal["allsum", "channelssum"] | None = "allsum",
        force_sampling: bool = False,
        channels_first: bool = True,
    ):
        return self.sample(z_mu, z_sigma, force_sampling), self.kl_divergence(
            z_mu, z_sigma, prior_mu, prior_sigma, kl_divergence_reduction, channels_first
        )

In [ ]:
test = GaussianLatentSpace()
display(test)

sample_input_mu = torch.randn(2, 16, 16, 16, 4)
sample_input_sigma = torch.rand(2, 16, 16, 16, 4)
o = test(sample_input_mu, sample_input_sigma, channels_first=False)
o[0].shape, o[1]

GaussianLatentSpace()

(torch.Size([2, 16, 16, 16, 4]), tensor(19114.0234))

# nbdev

In [ ]:
!nbdev_export